# AutoEncoders

> Collection of Autoencoder models

In [ ]:
#| default_exp models.autoencoders

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export

import torch.nn as nn
import torch
from lightning import LightningModule
from nimrod.image.datasets import ImageDataset
from torch.utils.data import DataLoader
from nimrod.modules import Encoder, Decoder

## Overview
A flexible, powerful autoencoder implementation using PyTorch and PyTorch Lightning, designed for representation learning, dimensionality reduction, and generative modeling.

## Core Components

### AutoEncoder Class
A modular autoencoder with configurable encoder and decoder:
- Supports custom encoder and decoder architectures
- Simple forward pass for data reconstruction
- Flexible input tensor handling

#### Key Parameters
- `encoder`: Custom encoder layer
- `decoder`: Custom decoder layer

### AutoEncoderPL Class
A PyTorch Lightning wrapper for autoencoder training:
- Integrated loss computation
- Standardized training, validation, and test steps
- Automatic logging of reconstruction loss
- Adam optimizer with configurable learning rate

## Features
- Modular design with separate encoder and decoder
- PyTorch Lightning integration
- Mean Squared Error (MSE) reconstruction loss
- Supports batch processing
- Easy to extend and customize

## Architectural Variants
- Standard Autoencoders
- Variational Autoencoders (VAE)
- Denoising Autoencoders
- Sparse Autoencoders
- Convolutional Autoencoders

## Supported Operations
- Data reconstruction
- Representation learning
- Dimensionality reduction
- Feature extraction
- Batch prediction
- Model evaluation

## Dependencies
- PyTorch
- PyTorch Lightning
- Nimrod custom modules (Encoder, Decoder)

## Usage Example
```python
# Create encoder and decoder
enc = Encoder()
dec = Decoder()

# Instantiate autoencoder
autoencoder = AutoEncoder(enc, dec)

# Wrap with Lightning module
pl_model = AutoEncoderPL(autoencoder)

# Train using PyTorch Lightning Trainer
trainer.fit(pl_model)
```

In [ ]:
#| export
class AutoEncoder(nn.Module):
    """ A modular autoencoder with configurable encoder and decoder """
    def __init__(self,
        encoder:nn.Module, # Encoder layer
        decoder:nn.Module # Decoder layer
        ):

        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(
        self,
        x:torch.Tensor # Tensor B x C X H X W
        )->torch.Tensor: # Reconstructed input tensor of shape B x C X H X W

        """
        Forward pass of the AutoEncoder model.
        """

        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

In [ ]:
show_doc(AutoEncoder.forward)

---

### AutoEncoder.forward

>      AutoEncoder.forward (x:torch.Tensor)

*Forward pass of the AutoEncoder model.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| x | Tensor | Tensor B x C X H X W |
| **Returns** | **Tensor** | **Reconstructed input tensor of shape B x C X H X W** |

In [ ]:
enc = Encoder()
dec = Decoder()
a = AutoEncoder(enc, dec)
batch = torch.rand((10, 28*28))
y = a(batch)
print(y.shape)

torch.Size([10, 784])


In [ ]:
ds = ImageDataset(name='mnist', data_dir='../data/image/')
dl = DataLoader(ds)
b = next(iter(dl))
print(len(b), b[0].shape, b[1].shape)


[17:14:02] INFO - mnist Dataset: init


2 torch.Size([1, 1, 28, 28]) torch.Size([1])


In [ ]:
#| export
class AutoEncoderPL(LightningModule):
    """ LightningModule for AutoEncoder """
    def __init__(
        self,
        autoencoder:AutoEncoder # AutoEncoder instance
        ):
        super().__init__()
        # self.save_hyperparameters()
        self.save_hyperparameters(ignore=['autoencoder'])
        self.autoencoder = autoencoder
        self.metric = torch.nn.MSELoss()

    def forward(
        self,
        x: torch.Tensor # Tensor B x L
        )->torch.Tensor: # Reconstructed input tensor of shape B x L
        """
        Forward pass of the AutoEncoder model.
        """
        return self.autoencoder(x)
    
    def predict_step(self, batch, batch_idx):
        """
        Forward pass of the AutoEncoder model.
        """
        x, y = batch
        x = x.view(x.size(0), -1)
        with torch.no_grad():
            return self.autoencoder(x)

    def _shared_eval(self, batch, batch_idx, prefix, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        x, _ = batch
        x = x.view(x.size(0), -1) # flatten B x C x H x W to B x L (grey pic)
        x_hat = self.autoencoder(x)
        loss = self.metric(x_hat, x)
        self.log(f"{prefix}/loss", loss, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)
        return loss

    def training_step(self, batch, batch_idx):
        return self._shared_eval(batch, batch_idx, "train")
    
    def test_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "test")
    
    def validation_step(self, batch, batch_idx):
        return self._shared_eval(batch, batch_idx, "val")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [ ]:
show_doc(AutoEncoderPL.forward)

---

### AutoEncoderPL.forward

>      AutoEncoderPL.forward (x:torch.Tensor)

*Forward pass of the AutoEncoder model.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| x | Tensor | Tensor B x L |
| **Returns** | **Tensor** | **Reconstructed input tensor of shape B x L** |

In [ ]:
#| hide
# def on_validation_batch_end(
#         self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx, wadn
#         ):
#         # `outputs` comes from `LightningModule.validation_step`
#         # which corresponds to our model predictions in this case
        
#         # Let's log 20 sample image predictions from the first batch
#         if batch_idx == 0:
#             n = 20
#             x, y = batch
#             images = [img for img in x[:n]]
#             captions = [f'Ground Truth: {y_i} - Prediction: {y_pred}' 
#                 for y_i, y_pred in zip(y[:n], outputs[:n])]
            
            
#             # Option 1: log images with `WandbLogger.log_image`
#             wandb_logger.log_image(
#                 key='sample_images', 
#                 images=images, 
#                 caption=captions)


In [ ]:
autoencoder_pl = AutoEncoderPL(a)
b = torch.rand((5,28*28))
y = autoencoder_pl(b)
print(y.shape)

torch.Size([5, 784])


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()